In [1]:
import xmltodict

with open('Tags.fods') as fd:
    doc = xmltodict.parse(fd.read())

In [2]:
descr = list()
name = list()
tags = list()
contr = list()
rarity = list()
for n,i in enumerate(doc['office:document']['office:body']['office:spreadsheet']['table:table']['table:table-row']):
    if i['table:table-cell'][0] is not None and 'text:p' in i['table:table-cell'][0]:
        if i['table:table-cell'][0]['text:p'].startswith('//') or n == 0:
            # Filter out entries with group names. Group affiliation makes zero difference for rocket building
            continue
        else:
            # Since it's parsed as ordered list, a couple entry orders were offset by one
            o = 1
            if i['table:table-cell'][1] is None:
                o = 0
            name.append(i['table:table-cell'][o]['text:p'])
            tags.append(i['table:table-cell'][o+2]['text:p'])
            rarity.append(i['table:table-cell'][o+4]['text:p'])
            if i['table:table-cell'][o+3] is not None:
                contr.append(True)
            else:
                contr.append(False)   
    else:
        # Different nestings including a hack for no 762
        if isinstance(i['table:table-cell'][1]['text:p'],str):
            descr.append(i['table:table-cell'][1]['text:p'])
        elif n == 762:
            descr.append(i['table:table-cell'][1]['text:p']['text:span'][0]['#text'])
        else:
            descr.append(i['table:table-cell'][1]['text:p']['text:span']['#text']+" "+i['table:table-cell'][1]['text:p']['#text'])
    if n >= 825:
        # The end of the relevant bit
        break

In [43]:
import pandas as pd

In [44]:
# Initial df
df = pd.DataFrame(list(zip(name,rarity,tags,contr,descr)),columns=['Name','Rarity','Tags','Controversial','Description'])
# String format
df.loc[df['Name'] == 'KÃ¡rmÃ¡n Line','Name'] = 'Karman Line'

In [45]:
# Add extra column for each tag for searchability
t = ['COOL','WEIRD','CLEVER','FUNNY','AWESOME','CREEPY','CUTE','NOOB','GROSS','NAUGHTY','WILD','ASPIRING','NERDY']
for i in t:
    df[i] = False
    df.loc[df['Tags'].str.contains(i,case=False),i] = True

In [46]:
# Remove false positives for controversial
fc = ['Under Pressure','Timed','Button','Split','Steer']
for i in fc:
    df.loc[df['Name'] == i,'Controversial'] = False

In [49]:
df

,Name,Rarity,Tags,Controversial,Description,COOL,WEIRD,CLEVER,FUNNY,AWESOME,CREEPY,CUTE,NOOB,GROSS,NAUGHTY,WILD,ASPIRING,NERDY
0,Symmetric,0,COOL,False,For having a symmetric rocket,True,False,False,False,False,False,False,False,False,False,False,False,False
1,Perfectly Symmetric,1,COOL,False,For having a perfectly symmetric rocket,True,False,False,False,False,False,False,False,False,False,False,False,False
2,Unsymmetric,1,WEIRD,False,For having a unsymmetric rocket,False,True,False,False,False,False,False,False,False,False,False,False,False
3,Extremely Unsymmetric,2,WEIRD,False,For having a completely unsymmetric rocket,False,True,False,False,False,False,False,False,False,False,False,False,False
4,Thin,1,CLEVER,False,For having the rocket width at least 5 times s...,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,RaySon,1,CLEVER,False,For beating a challenge from RaySon,False,False,True,False,False,False,False,False,False,False,False,False,False
397,Shop,0,CLEVER,False,For beating a challenge from DIY Shop,False,False,True,False,False,False,False,False,False,False,False,False,False
398,Science,1,CLEVER,False,For beating a challenge from dot_science,False,False,True,False,False,False,False,False,False,False,False,False,False
399,Family,2,"COOL, WEIRD",False,For beating a challenge from ScrapDad,True,True,False,False,False,False,False,False,False,False,False,False,False


In [132]:
df.to_csv("Tags1_5.csv")

In [133]:
df.to_pickle("Tags1_5.pickle")